In [1]:
import torch
import numpy
from timeit import timeit

In [2]:
class CustomDropout(torch.nn.Module):
    def __init__(self, p):
        super().__init__()
        self.p = p
        
    def forward(self, input):
        # Here we have to specify choice option probabilities
        if self.p != 0.5:
            mask = torch.from_numpy(numpy.random.choice([0, 1], size=input.shape, p=[self.p, 1 - self.p])).float()
            
        # This is default with equal chance for every option
        else:
            mask = torch.from_numpy(numpy.random.choice([0, 1], size=input.shape)).float()
        return input * mask
    
class GaussianDropout(torch.nn.Module):
    def __init__(self, p):
        super(GaussianDropout, self).__init__()
        self.alpha = torch.Tensor([p/(1-p)])
        
    def forward(self, x):
        """
        Sample noise   e ~ N(1, alpha)
        Multiply noise h = h_ * e
        """
        if self.train():
            # N(1, alpha)
            epsilon = torch.randn(x.size()) * self.alpha + 1

            #epsilon = Variable(epsilon)
            if x.is_cuda:
                epsilon = epsilon.cuda()

            return x * epsilon
        else:
            return x

In [3]:
p1 = 0.5
test_tensor = torch.rand(10, 50000)
std_dropout = torch.nn.Dropout(p1)
custom_dropout = CustomDropout(p1)
gaussian_dropout = GaussianDropout(p1)

### *Dropout with p=0.5*

(Fast for numpy's choice)


In [4]:
timeit(
    "std_dropout(test_tensor)", setup="from __main__ import test_tensor, std_dropout", number=10000
)

61.208337502999996

In [5]:
timeit(
    "custom_dropout(test_tensor)", setup="from __main__ import test_tensor, custom_dropout", number=10000
)

41.747105727999994

In [6]:
timeit(
    "gaussian_dropout(test_tensor)", setup="from __main__ import test_tensor, gaussian_dropout", number=10000
)

34.54468514899999

### *Dropout with other p*

(No speed gains)

In [7]:
p2 = 0.3
test_tensor2 = torch.rand(10, 50000)
std_dropout2 = torch.nn.Dropout(p2)
custom_dropout2 = CustomDropout(p2)
gaussian_dropout2 = GaussianDropout(p2)

In [8]:
timeit(
    "std_dropout2(test_tensor2)", setup="from __main__ import test_tensor2, std_dropout2", number=10000
)

60.47110804799999

In [9]:
timeit(
    "custom_dropout2(test_tensor2)", setup="from __main__ import test_tensor2, custom_dropout2", number=10000
)

111.050683797

In [10]:
timeit(
    "gaussian_dropout2(test_tensor2)", setup="from __main__ import test_tensor2, gaussian_dropout2", number=10000
)

36.710593152